In [1]:
# Cell A — install
!pip install -q openai jsonschema

In [12]:
# Cell B — securely set the API key (hidden input)
from getpass import getpass
import os

key = getpass("Paste your GROQ API key (input hidden): ")
os.environ['GROQ_API_KEY'] = key
# base url for Groq's OpenAI-compatible endpoint (default)
os.environ['GROQ_BASE_URL'] = 'https://api.groq.com/openai/v1'
print("API key set in environment. (Hidden from view.)")

Paste your GROQ API key (input hidden): ··········
API key set in environment. (Hidden from view.)


In [4]:
# Cell C — quick check
import os
print('GROQ_API_KEY present?:', 'GROQ_API_KEY' in os.environ)
print('GROQ_BASE_URL:', os.environ.get('GROQ_BASE_URL'))

GROQ_API_KEY present?: True
GROQ_BASE_URL: https://api.groq.com/openai/v1


In [28]:
# ========================================
# Task 1: Conversation Manager with Groq API
# ========================================

# 1. Install dependencies
!pip install -q openai jsonschema

# 2. Imports and setup
import os
from typing import List, Dict, Any
from dataclasses import dataclass, field
from openai import OpenAI


# Create Groq client (OpenAI-compatible)
client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

# 3. Groq Chat Completion Helper
def groq_chat_completion(messages: List[Dict[str, str]],
                         model: str = "llama-3.1-8b-instant",
                         temperature: float = 0.2) -> Dict[str, Any]:
    """
    Calls Groq's OpenAI-compatible endpoint.
    Returns dict with assistant message.
    """
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature
        )
        return {
            "choices": [{
                "message": {
                    "role": "assistant",
                    "content": resp.choices[0].message.content
                }
            }]
        }
    except Exception as e:
        print("Warning: API call failed ->", e)
        return {
            "choices": [{
                "message": {"role": "assistant", "content": "SIMULATED SUMMARY (API not called)."}
            }]
        }

# 4. Conversation Manager Class
@dataclass
class ConversationManager:
    history: List[Dict[str, str]] = field(default_factory=list)
    summary_history: List[str] = field(default_factory=list)
    run_count: int = 0
    summarize_every_k: int = 3
    summarization_model: str = "llama-3.1-8b-instant"

    def add_user_message(self, content: str):
        self.history.append({"role": "user", "content": content})

    def add_assistant_message(self, content: str):
        self.history.append({"role": "assistant", "content": content})

    def get_last_n_turns(self, n: int) -> List[Dict[str, str]]:
        return self.history[-n:]

    def truncate_by_chars(self, max_chars: int) -> None:
        total = sum(len(m["content"]) for m in self.history)
        while total > max_chars and self.history:
            removed = self.history.pop(0)
            total -= len(removed["content"])

    def truncate_by_words(self, max_words: int) -> None:
        def total_words(hist):
            return sum(len(m["content"].split()) for m in hist)
        while total_words(self.history) > max_words and self.history:
            self.history.pop(0)

    def maybe_summarize(self):
        self.run_count += 1
        if self.summarize_every_k > 0 and (self.run_count % self.summarize_every_k == 0):
            print(f"Summarizing at run {self.run_count}...")
            summary = self.summarize_history()
            self.summary_history.append(summary)
            # Replace history with the summary only
            self.history = [{"role": "assistant", "content": f"[SUMMARY]\n{summary}"}]
            return summary
        return None

    def summarize_history(self) -> str:
        messages = [
            {"role": "system", "content": "You are a concise summarizer. Produce a short summary."},
            {"role": "user", "content": "Summarize the conversation below."}
        ]
        convo_text = "\n".join([f"{m['role']}: {m['content']}" for m in self.history])
        messages.append({"role": "user", "content": convo_text})

        resp = groq_chat_completion(messages=messages, model=self.summarization_model)
        choices = resp.get("choices", [])
        if choices:
            return choices[0]["message"].get("content", "")
        return "[EMPTY SUMMARY]"

    def show_history(self):
        print("--- Conversation History ---")
        for i, m in enumerate(self.history, 1):
            print(f"{i}. {m['role'].upper()}: {m['content']}")
        print("---------------------------")

# 5. Demonstration
cm = ConversationManager(summarize_every_k=3)

# Feed conversation (summary triggers on 3rd run)
sample_turns = [
    ("Hi, I need help booking a flight to Bangalore next Friday.",
     "Sure — what is your preferred airline and departure city?"),
    ("Preferably early morning. My departure city is Mumbai.",
     "Got it. Do you have a budget in mind?"),
    ("Budget under 8k INR. Also, I have a connecting flight constraint.",
     "I'll look for morning flights under 8k with max one connection.")
]

print("\nFeeding sample conversation...")
for user_msg, assistant_msg in sample_turns:
    cm.add_user_message(user_msg)
    cm.add_assistant_message(assistant_msg)
    summary = cm.maybe_summarize()
    if summary:
        print("\n*** SUMMARY GENERATED ***\n", summary)

print("\nFinal history after summarization:")
cm.show_history()

# Truncation demo
print("\nDemonstrating truncation...")
for i in range(6):
    cm.add_user_message(f"User message number {i} with extra text.")
    cm.add_assistant_message(f"Assistant message number {i} with details.")

print("\nHistory length before truncation:", len(cm.history))

# Last 4 messages
last_4 = cm.get_last_n_turns(4)
print("\nLast 4 messages:")
for m in last_4:
    print(m)

# Truncate by chars
cm_copy = ConversationManager()
cm_copy.history = cm.history.copy()
cm_copy.truncate_by_chars(200)
print("\nHistory after truncating to 200 chars:", len(cm_copy.history))

# Truncate by words
cm_copy2 = ConversationManager()
cm_copy2.history = cm.history.copy()
cm_copy2.truncate_by_words(50)
print("\nHistory after truncating to 50 words:", len(cm_copy2.history))



Feeding sample conversation...
Summarizing at run 3...

*** SUMMARY GENERATED ***
 The user needs to book a flight from Mumbai to Bangalore next Friday, with a preferred early morning departure, a budget under 8k INR, and a maximum of one connecting flight.

Final history after summarization:
--- Conversation History ---
1. ASSISTANT: [SUMMARY]
The user needs to book a flight from Mumbai to Bangalore next Friday, with a preferred early morning departure, a budget under 8k INR, and a maximum of one connecting flight.
---------------------------

Demonstrating truncation...

History length before truncation: 13

Last 4 messages:
{'role': 'user', 'content': 'User message number 4 with extra text.'}
{'role': 'assistant', 'content': 'Assistant message number 4 with details.'}
{'role': 'user', 'content': 'User message number 5 with extra text.'}
{'role': 'assistant', 'content': 'Assistant message number 5 with details.'}

History after truncating to 200 chars: 5

History after truncating to

In [29]:
# ========================================
# Task 2: Conversation Classification with Groq API
# ========================================

# 1. Define possible intents (fixed set of categories)
INTENTS = [
    "book_flight",
    "cancel_flight",
    "check_weather",
    "small_talk",
    "other"
]

# 2. Intent classification helper
def classify_intent(user_message: str,
                    model: str = "llama3-8b-8192") -> str:
    """
    Uses Groq LLM to classify a user message into one of the predefined intents.
    Falls back to 'other' if API call fails.
    """
    messages = [
        {"role": "system", "content": f"You are an intent classifier. Classify the user message into one of these intents: {INTENTS}. Respond with only the intent name."},
        {"role": "user", "content": user_message}
    ]
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.0  # deterministic classification
        )
        intent = resp.choices[0].message.content.strip()
        if intent not in INTENTS:
            intent = "other"
        return intent
    except Exception as e:
        print("Warning: Classification API failed ->", e)
        return "other"

# 3. Extend ConversationManager to store intents
@dataclass
class ClassifiedConversationManager(ConversationManager):
    intents: List[str] = field(default_factory=list)

    def add_user_message(self, content: str):
        # classify intent before storing
        intent = classify_intent(content, model=self.summarization_model)
        self.history.append({"role": "user", "content": content, "intent": intent})
        self.intents.append(intent)

    def show_history(self):
        print("--- Classified Conversation History ---")
        for i, m in enumerate(self.history, 1):
            role = m['role'].upper()
            content = m['content']
            intent = m.get('intent', 'N/A')
            if role == "USER":
                print(f"{i}. {role}: {content}  [Intent: {intent}]")
            else:
                print(f"{i}. {role}: {content}")
        print("--------------------------------------")

# 4. Demonstration
ccm = ClassifiedConversationManager(summarize_every_k=3)

sample_turns = [
    ("Hi, I need help booking a flight to Bangalore next Friday.",
     "Sure — what is your preferred airline and departure city?"),
    ("Preferably early morning. My departure city is Mumbai.",
     "Got it. Do you have a budget in mind?"),
    ("What's the weather like in Bangalore on that day?",
     "Let me fetch the forecast for you.")
]

print("\nFeeding classified conversation...")
for user_msg, assistant_msg in sample_turns:
    ccm.add_user_message(user_msg)
    ccm.add_assistant_message(assistant_msg)
    summary = ccm.maybe_summarize()
    if summary:
        print("\n*** SUMMARY GENERATED ***\n", summary)

print("\nFinal classified history:")
ccm.show_history()

print("\nDetected intents:", ccm.intents)



Feeding classified conversation...
Summarizing at run 3...

*** SUMMARY GENERATED ***
 A user is seeking assistance in booking a flight from Mumbai to Bangalore for next Friday. They prefer an early morning departure and are open to discussing airline and budget options, but the conversation was cut off before further details were discussed.

Final classified history:
--- Classified Conversation History ---
1. ASSISTANT: [SUMMARY]
A user is seeking assistance in booking a flight from Mumbai to Bangalore for next Friday. They prefer an early morning departure and are open to discussing airline and budget options, but the conversation was cut off before further details were discussed.
--------------------------------------

Detected intents: ['book_flight', 'book_flight', 'check_weather']
